In [ ]:
from langchain_groq.chat_models import ChatGroq
from langchain.prompts import ChatPromptTemplate
import os
from utils import get_completion
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
chat = ChatGroq(
    api_key=os.environ['GROQ_API_KEY'], 
    temperature=0.0,
    model="openai/gpt-oss-120b"
)
print(chat)

client=<groq.resources.chat.completions.Completions object at 0x000001FBF60B30E0> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FBF60B3E00> model_name='openai/gpt-oss-120b' temperature=1e-08 model_kwargs={} groq_api_key=SecretStr('**********')


**The ChatPromptTemplate object has useful methods, so its used instead of regular string/fstring variable**

In [ ]:
template_string = """Translate the text that is delimited by triple backticks into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [ ]:
customer_style = """American English, in a calm and respectful tone"""
customer_email = """
Arrr, I be fuming that my blender lid flew off and splattered me kitchen walls with smoothie! \
And to make matters worst, the warranty dont cover the cost of cleaning up me kitchen. I need your help rignt now. mate!
"""

customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])
print("\nTRANSFORMED CUSTOMER MESSAGE: ", get_completion(customer_messages))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content='Translate the text that is delimited by triple backticks into a style that is American English, in a calm and respectful tone. text: ```\nArrr, I be fuming that my blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worst, the warranty dont cover the cost of cleaning up me kitchen. I need your help rignt now. mate!\n```\n' additional_kwargs={} response_metadata={}

TRANSFORMED CUSTOMER MESSAGE:  I’m quite upset that the lid on my blender came off and splattered my kitchen walls with smoothie. Unfortunately, the warranty does not cover the cost of cleaning the kitchen. I would appreciate your help right away.


In [ ]:
service_reply = """Hey there customer, the warranty does not cover cleaning expenses for your kitchen \
because its your fault that you misused your blender by forgetting to put the lid on before starting the blender. \
Tough luck! See ya!"""

service_style_pirate = """a polite tone that speaks in English Pirate"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate, text=service_reply
)
print(service_messages[0].content)

service_response = get_completion(service_messages)
print(service_response)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because its your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!```

Ahoy, esteemed customer! I regret to inform ye that the warranty doth not extend to cover the cleaning expenses for yer galley, as the mishap arose from the misuse of yer blender—specifically, the lid was left ajar afore the blades set to work. I understand this be a disappointment, and I thank ye kindly for yer understanding. Fair winds to ye!


### LangChain's Output Parser

In [ ]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable'}

In [ ]:
customer_review = """\
This leafblower is pretty amazing. It has 4 settings: candle blower, gentle breeze, wind city and tornado. \
It arrived in 2 days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. \
Its slightly more expensive than the other leafblowers out there but I think its worth it for the extra features.
"""

review_template = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer with True or False as values.
delivery_days: How many days did it take for the product to get delivered? Answer with Integer.
price_value:  Extract any sentences about the value or price,

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

messages = prompt_template.format_messages(text=customer_review)
chat = ChatGroq(temperature=0.0, model="openai/gpt-oss-120b")
response = chat(messages)
print(response.content)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nFor the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer with True or False as values.\ndelivery_days: How many days did it take for the product to get delivered? Answer with Integer.\nprice_value:  Extract any sentences about the value or price,\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


C:\Users\fredr\AppData\Local\Temp\ipykernel_7596\3825791206.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)


{
  "gift": true,
  "delivery_days": 2,
  "price_value": "Its slightly more expensive than the other leafblowers out there but I think its worth it for the extra features."
}


In [ ]:
type(response.content)

str

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response.content.get('gift')

**Preparing the Output Parser**

In [ ]:
gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer with True or False as values.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to get delivered? Answer with Integer.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price")
response_schema = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser(response_schemas=response_schema)

format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift for someone else? Answer with True or False as values.\n\t"delivery_days": string  // How many days did it take for the product to get delivered? Answer with Integer.\n\t"price_value": string  // Extract any sentences about the value or price\n}\n```'

In [ ]:
review_template_2 = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer with True or False as values.
delivery_days: How many days did it take for the product to get delivered? Answer with Integer.
price_value:  Extract any sentences about the value or price,

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions
)
print(messages[0].content)


For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer with True or False as values.
delivery_days: How many days did it take for the product to get delivered? Answer with Integer.
price_value:  Extract any sentences about the value or price,

text: This leafblower is pretty amazing. It has 4 settings: candle blower, gentle breeze, wind city and tornado. It arrived in 2 days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. Its slightly more expensive than the other leafblowers out there but I think its worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? 

In [ ]:
chat = ChatGroq(model="openai/gpt-oss-120b", temperature=0.0)
response = chat(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": "Its slightly more expensive than the other leafblowers out there but I think its worth it for the extra features."
}
```


In [ ]:
output_dict = output_parser.parse(response.content)
print(type(output_dict))
output_dict

<class 'dict'>


{'gift': True,
 'delivery_days': 2,
 'price_value': 'Its slightly more expensive than the other leafblowers out there but I think its worth it for the extra features.'}